# Notebook for testing without publishing

This notebook is for testing the creation of a DownholeCollection (intermediary object) and conversion to an Evo Downhole Collection.

It will not publish the result to Evo, for that you will need `packages/gef/samples/convert-gef/convert-gef.ipynb`

This notebook may be deleted prior to PR to upstream, it's main purpose is testing features as they are developed.

In [ ]:
import os
from evo.notebooks import ServiceManagerWidget

# Credentials can be provided from .env or filled into second params below.
# Use `uv run --env-file .env` to load environment variables from .env file.
client_id = os.getenv("EVO_CLIENT_ID", "")
base_uri = os.getenv("EVO_BASE_URI", "")
discovery_url = os.getenv("EVO_DISCOVERY_URL", "")

manager = await ServiceManagerWidget.with_auth_code(
    client_id=client_id, base_uri=base_uri, discovery_url=discovery_url
).login()

In [ ]:
from evo.data_converters.common import create_evo_object_service_and_data_client

object_service_client, data_client = create_evo_object_service_and_data_client(service_manager_widget=manager)

In [ ]:
import glob
import pandas as pd
from pprint import pp
from evo.data_converters.gef.converter import create_from_parsed_gef_cpts, parse_gef_files
from evo.data_converters.common.objects import DownholeCollectionToGeoscienceObject

# Data files in data/input have been synthentically generated from a python script.
gef_files = glob.glob("data/input/*.gef")
parsed_cpt_files = parse_gef_files(gef_files)
# pprint(parsed_cpt_files)

downhole_collection = create_from_parsed_gef_cpts(parsed_cpt_files)

# Manually add an interval table
interval_table_data = {
    # Hole index will come from LOCA_ID and mapped to the index of the hole_id in the collars df using 1-based arrays
    "hole_index": [1, 2, 3, 4, 5],
    "GEOL_TOP": [6.00, 6.00, 6.00, 6.00, 6.00],
    "GEOL_BASE": [7.00, 7.00, 7.00, 7.00, 7.00],
    "GEOL_DENSITY": [1.23, 1.23, 1.23, 1.23, 1.23],
}
interval_table = pd.DataFrame(interval_table_data)
downhole_collection.add_measurement_table(input=interval_table)

converter = DownholeCollectionToGeoscienceObject(dhc=downhole_collection, data_client=data_client)
geoscience_object = converter.convert()

# pp(downhole_collection.collars.get_attributes_df())

# attributes_df = downhole_collection.collars.get_attributes_df()

# for k, v in attributes_df.items():
#     pp(type(k))
#     pp(v.name)
pp(geoscience_object)